In [1]:
import bempp.api
from bempp.api.assembly.blocked_operator import BlockedOperator
import math
import numpy as np
import scipy 
import cmath
from numba import objmode
from numpy.linalg import slogdet
from bempp.api.operators.far_field import helmholtz as helmholtz_farfield
import matplotlib.pyplot as plt

In [ ]:
h = 0.1

dist = 0.5

r1_equal = 0.5
r2_equal = 1

sphere1 = bempp.api.shapes.sphere(r = r1_equal, h=h, origin=((dist/2) + r1_equal, 0, 0))
sphere2 = bempp.api.shapes.sphere(r = r2_equal, h=h, origin=(-((dist/2) + r2_equal), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

In [ ]:
#grid1 = bempp.api.export('sphere1.msh', grid = sphere1)
#grid2 = bempp.api.export('sphere2.msh', grid = sphere2)

In [ ]:
N_wn = 10
lbound_wn = 0
ubound_wn = 4
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

In [ ]:
err_005_30 = [0.12665240336518885, 0.33322649361970363, 0.4324803657198988, 0.5310368394331934, 0.6384288571389283, 0.7620902752257449, 0.9109753971321096, 1.0980605626635076, 1.3441326029816862, 1.6854801705411153]
err_005_15 = [0.17786594808214767, 0.293032809221281, 0.3700122521715969, 0.443680713524899, 0.5218480663887473, 0.6100712320286158, 0.7145711366886984, 0.8438903208700517, 1.0110375929440119, 1.2372940468028604]
err_005_05 = [0.13654558584023216, 0.21198319874244248, 0.2611810976373154, 0.30692619059491366, 0.3535053820313234, 0.4033983841504024, 0.45900072952789106, 0.5233681706514199, 0.6009875213242425, 0.6989876680126172]

In [ ]:
err_01_30 = [0.03542620249755996, 0.3210786299199821, 0.42247309264912375, 0.5223726001011625, 0.6307488440438795, 0.7551850921567027, 0.9046979084946863, 1.0922844934788665, 1.3387194629363108, 1.6802332835470124]
err_01_15 = [0.15903425181750763, 0.2806629410923526, 0.35979869078517407, 0.4348207725021387, 0.5139814518264912, 0.6029874287531054, 0.7081260978252044, 0.8379704814848771, 1.0055362670200065, 1.2320851123943841]
err_01_05 = [0.11568041313682595, 0.19757713198697635, 0.24887522295692877, 0.29591902260073105, 0.3434459733520769, 0.3940901241983566, 0.45031839448032496, 0.5152245682780814, 0.593315925428474, 0.6917285768878242]

In [ ]:
x = -np.log(quad_points)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))
ax1.semilogy(x, err_005_05, '^-')
ax1.semilogy(x, err_01_05, 'o-')
ax1.legend([ 'Error ($h = 0.05$)', 'Error ($h = 0.1$)'], fontsize = 15)

ax1.set_xlabel('$1\mathrm{i}*k$', fontsize = 15)
ax1.set_ylabel('relative error', fontsize = 15)
ax1.set_title('$Z = 0.5$', fontsize = 15)
    
ax2.semilogy(x, err_005_15, '^-')
ax2.semilogy(x, err_01_15, 'o-')
ax2.legend([ 'Error ($h = 0.05$)', 'Error ($h = 0.1$)'], fontsize = 15)

ax2.set_xlabel('$1\mathrm{i}*k$', fontsize = 15)
ax2.set_ylabel('relative error', fontsize = 15)
ax2.set_title('$Z = 1.5$', fontsize = 15)

ax3.semilogy(x, err_005_30, '^-')
ax3.semilogy(x, err_01_30, 'o-')
ax3.legend([ 'Error ($h = 0.05$)', 'Error ($h = 0.1$)'], fontsize = 15)

ax3.set_xlabel('$1\mathrm{i}*k$', fontsize = 15)
ax3.set_ylabel('relative error', fontsize = 15)
ax3.set_title('$Z = 3.0$', fontsize = 15)

fig.tight_layout()

fig.suptitle('Relative error between the estimated integrand value and asymptotic series in two spheres with equal radii case', fontsize = 20)
fig.subplots_adjust(top=0.85)
fig.savefig('rel_err_equal.pdf')

# Unequal 

In [11]:
h = 0.1
r1_equal = 1
r2_equal = 1

for dist in [0.5, 0.75, 1, 1.25, 1.5, 1.75, 2.0, 2.25, 2.5, 2.75, 3.0]:

    sphere1 = bempp.api.shapes.sphere(r = r1_equal, h=h, origin=((dist/2) + r1_equal, 0, 0))
    sphere2 = bempp.api.shapes.sphere(r = r2_equal, h=h, origin=(-((dist/2) + r2_equal), 0, 0))

    space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
    space_sphere2 = bempp.api.function_space(sphere2, "P", 1)
    
    for wavenumber in [-np.log(quad_points)[9]*1j,-np.log(quad_points)[8]*1j,-np.log(quad_points)[7]*1j,-np.log(quad_points)[6]*1j,-np.log(quad_points)[5]*1j,-np.log(quad_points)[4]*1j,
                       -np.log(quad_points)[3]*1j,-np.log(quad_points)[2]*1j,-np.log(quad_points)[1]*1j,-np.log(quad_points)[0]*1j, 5j, 6j, 7j, 8j, 9j, 10j]:
        slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
        slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
        slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
        slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

        mat11 = slp11.weak_form().A
        mat12 = slp12.weak_form().A
        mat21 = slp21.weak_form().A
        mat22 = slp22.weak_form().A
        mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
        mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

        mat = np.block([[mat11,mat12],[mat21,mat22]])  
        mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
        mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

        inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                              [mat21_zero, mat22_inv]])  

        combined_inverse_free = inv_free_mat2.dot(mat)
        sign, logdet = slogdet(combined_inverse_free)

        asy = (-(r1_equal*r2_equal)/(4*dist*(r1_equal+r2_equal+dist)))*np.exp(2*1j*dist*wavenumber)
        print(dist, wavenumber, logdet/asy)

0.5 (-0+0j) (1.6917285768878243-0j)
0.5 0.11549619760378992j (1.593315925428474-0j)
0.5 0.24609503336885402j (1.5152245682780812-0j)
0.5 0.39634896722901447j (1.4503183944803248-0j)
0.5 0.5732404646099698j (1.3940901241983565-0j)
0.5 0.7882938150967403j (1.3434459733520767-0j)
0.5 1.0626359889199166j (1.295919022600731-0j)
0.5 1.4419435745072013j (1.2488752229569287-0j)
0.5 2.060488851853035j (1.1975771319869764-0j)
0.5 4j (1.1156804131368259-0j)
0.5 5j (1.0915192552102089-0j)
0.5 6j (1.072606190373997-0j)
0.5 7j (1.0570198552269665-0j)
0.5 8j (1.0437153540855855-0j)
0.5 9j (1.0320659974702158-0j)
0.5 10j (1.0216668836005731-0j)
0.75 (-0+0j) (1.859047606719003-0j)
0.75 0.11549619760378992j (1.7224797752948533-0j)
0.75 0.24609503336885402j (1.6171135093246205-0j)
0.75 0.39634896722901447j (1.5321013492274458-0j)
0.75 0.5732404646099698j (1.4606588312981772-0j)
0.75 0.7882938150967403j (1.3982116086695824-0j)
0.75 1.0626359889199166j (1.3412471637387648-0j)
0.75 1.4419435745072013j (1.28

In [ ]:
np.exp(-2*1.5*15)

In [3]:
N_wn = 10
lbound_wn = 0
ubound_wn = 4
quad_points = np.linspace(np.exp(-ubound_wn), np.exp(-lbound_wn), N_wn)

In [8]:
h = 0.1

dist = 1.5

r1_equal = 1
r2_equal = 0.5

sphere1 = bempp.api.shapes.sphere(r = r1_equal, h=h, origin=((dist/2) + r1_equal, 0, 0))
sphere2 = bempp.api.shapes.sphere(r = r2_equal, h=h, origin=(-((dist/2) + r2_equal), 0, 0))

space_sphere1 = bempp.api.function_space(sphere1, "P", 1)
space_sphere2 = bempp.api.function_space(sphere2, "P", 1)

In [9]:
#quotient_ = []
#for index2, point2 in enumerate(quad_points):
#    wavenumber = -np.log(point2)*1j

#for wavenumber in [30j, 35j, 40j, 45j]:
for wavenumber in [8j, 9j]:
    slp11 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere1, space_sphere1, wavenumber)
    slp12 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere1, space_sphere1, wavenumber)
    slp21 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere1, space_sphere2 , space_sphere2 , wavenumber)
    slp22 = bempp.api.operators.boundary.helmholtz.single_layer(space_sphere2 , space_sphere2 , space_sphere2 , wavenumber)

    mat11 = slp11.weak_form().A
    mat12 = slp12.weak_form().A
    mat21 = slp21.weak_form().A
    mat22 = slp22.weak_form().A
    mat12_zero = np.zeros((mat11.shape[0],mat12.shape[1]))
    mat21_zero = np.zeros((mat22.shape[0],mat11.shape[1]))

    mat = np.block([[mat11,mat12],[mat21,mat22]])  
    mat11_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat11), np.eye(mat11.shape[0]))
    mat22_inv = scipy.linalg.lu_solve(scipy.linalg.lu_factor(mat22), np.eye(mat22.shape[0]))

    inv_free_mat2 = np.block([[mat11_inv,  mat12_zero],
                          [mat21_zero, mat22_inv]])  

    combined_inverse_free = inv_free_mat2.dot(mat)
    sign, logdet = slogdet(combined_inverse_free)
    
    asy = (-(r1_equal*r2_equal)/(4*dist*(r1_equal+r2_equal+dist)))*np.exp(2*1j*dist*wavenumber)
    print(logdet/asy)

(0.4545076367405072-0j)
(-11.634037430759726-0j)


In [ ]:
print(quotient_)

In [ ]:
for i in reversed(quotient_):
    print(i)

In [ ]:
# h = 0.1, r1 = 1, r2 = 1, dist = 0.5

#quadrature points + 5j 10j 15j 20j 25j

quo_05_01 = [1.6917285768878243, 1.593315925428474, 1.5152245682780812, 1.4503183944803248, 1.3940901241983565, 1.3434459733520767, 
             1.295919022600731, 1.2488752229569287, 1.1975771319869764, 1.1156804131368259, 1.0915192552102089, 1.0216668836005731, 
             0.9810185111692122, 0.9494621500602526, 1.0185745856842798]
quo_05_005 = [1.6989876680126172, 1.6009875213242424, 1.52336817065142, 1.459000729527891, 1.4033983841504023, 1.3535053820313234, 
              1.3069261905949137, 1.2611810976373155, 1.2119831987424423, 1.136545585840232, 1.1156269481455499, 1.0611344272628642,
              1.034713444552717, 1.0231496635293067, 1.2536825287441042]

# h = 0.1, r1 = 1, r2 = 1, dist = 1.5

#quadrature points + 5j 6j 7j 8j 9j 10j 15j 20j 25j

quo_15_01 = [2.232085112394384, 2.0055362670200068, 1.8379704814848772, 1.7081260978252042, 1.6029874287531054, 1.5139814518264911, 
             1.4348207725021387, 1.359798690785174, 1.2806629410923527, 1.1590342518175076, 1.1250566858548308, 1.0994675921230708, 
             1.0565948952482926, 0.5074700046136563, -11.996427099709702, -174.45607130201157, 87638206.560096, -734864927950574, 
             2.1490016909545717e+21]
quo_15_005 = [2.23729404680286, 2.011037592944012, 1.8438903208700517, 1.7145711366886982, 1.610071232028616, 1.5218480663887473,
              1.443680713524899 ,1.370012252171597, 1.293032809221281, 1.1778659480821476, 1.1470881857694344, 1.124446025077855, 
              1.0896733477921663, 0.9066772712343802, 0.274140253235022, -96.96986996678503, -2642992381.854194, 5799575311818628, 
              6.042272069099488e+21]

# h = 0.1, r1 = 1, r2 = 1, dist = 3.0

#quadrature points + 5j 10j 15j 20j 25j
quo_30_01 = [2.6802332835470124, 2.338719462936311, 2.0922844934788665, 1.9046979084946862, 1.7551850921567027, 1.6307488440438795, 
             1.5223726001011626, 1.4224730926491238, 1.3210786299199822, 1.03542620249756, 0.7830484287444888, -9649.43991264451, 
             -92872562.5153004, -42506129539.232056, -19199795176356.457, -4587200326647995, 7.113329142570317e+27, -2.3586806406941405e+41, 
             2.3367754494989145e+54]
quo_30_005 = [2.6854801705411155, 2.344132602981686, 2.0980605626635076, 1.9109753971321095, 1.7620902752257448, 1.6384288571389283, 
              1.5310368394331935, 1.4324803657198988, 1.3332264936197038, 1.1266524033651888, 85.565837395448, -26306.211190424227, 
              -37121218.84967844, -15359043508.457905, 1312419705631.9238, -3039876037360761, -2.666726124156957e+29, 1.8960664818839644e+42, 
              6.634325531235298e+54]

In [ ]:
# h = 0.1, r1 = 0.5, r2 = 1, dist = 0.5

#quadrature points + 5j 10j 15j 20j 25j

quo_05_01_un = [1.8206545702975787, 1.7273911666409547, 1.6472709073414826, 1.5766032126980472, 1.512603490261483, 1.4529576306431395, 1.39538260284737, 
             1.3368635407942269, 1.271095420144403, 1.1607761852675293, 1.1269577033221698, 1.027820709172044, 0.9703399961428494, 0.9266587052987264, 
             0.9851350574347065]

quo_05_005_un = [1.8345052560377437, 1.7417353319115643, 1.662227075340491, 1.5922974531104097, 1.529187631027774, 1.4706357282569131, 1.4144612696610703, 
              1.3578753865685909, 1.2952351848512098, 1.1944156216367312, 1.217349118298432, 1.0276962292986367, 1.0496007646766177, 1.087717746895201,
              1.1652848573339245]
# h = 0.1, r1 = 0.5, r2 = 1, dist = 1.5

#quadrature points + 5j 6j 7j 8j 9j 10j 15j 20j 25j

quo_15_01_un = [2.4247737475139273, 2.2199059113006405, 2.0537751322444366, 1.9148248353735333, 1.7949467809428923,  1.6880213428005857, 1.5888046097471653, 
             1.491538331995559, 1.3861168223597011, 1.219688514705622, 1.1723242365831077, 1.1364324476357923, 1.090157844819815, 0.7022476483810213, 
             -9.503292501931128, -146.84293625567767, -54035028.04900226, -608431645315599.2, 1.8636369463650492e+21]
quo_15_005_un = [2.4367705036042397, 2.232070620475216, 2.0663530318886405, 1.9280294868711763, 1.8089933491579089, 1.703161587525873, 1.605382284587698,
              1.510116002298733, 1.407915796374946, 1.2511387137687873, 1.208523537473988, 1.1775586330642027, 1.1547756043189783, 1.1785648758898601, 
              8.737755401753223, -17.639945512970833, -3066313310.036537, 3135727984484750, 4.722506753599183e+21]

# h = 0.1, r1 = 0.5, r2 = 1, dist = 3.0

#quadrature points + 5j 10j 15j 20j 25j
quo_30_01_un = [2.8708945847679415, 2.57295343520804, 2.336621280265407, 2.141963900944468, 1.9762357656836573, 1.8305559292803348, 1.6978809938062513, 
             1.571063089995682, 1.4383854283012583, 1.1596138237173579, 0.3844055922933073, -3164483568621943, 1.6535441686843786e+27, 
             -1.7795659021723287e+41, 2.6955080810787252e+54]
quo_30_005_un = [2.8837283563113227, 2.585631952992869, 2.3495014036045085, 2.1553188619263928, 1.990312963476716, 1.8456244050894977, 1.714293818957943, 
              1.589388699834989, 1.4598461983554125, 1.3346199088634219, -45.74426548278499, 342328382585691, -3.1136383027668546e+29, 
              1.1581252259655788e+42, 3.594567813019431e+54]

In [ ]:
0.5 (-0+0j) (1.8208179913424725-0j)
0.5 0.11549619760378992j (1.727576086121002-0j)
0.5 0.24609503336885402j (1.6474813240457988-0j)
0.5 0.39634896722901447j (1.5768442673589167-0j)
0.5 0.5732404646099698j (1.5128821096102028-0j)
0.5 0.7882938150967403j (1.4532837001318677-0j)
0.5 1.0626359889199166j (1.3957714026704766-0j)
0.5 1.4419435745072013j (1.3373418511297024-0j)
0.5 2.060488851853035j (1.2717228603423079-0j)
0.5 4j (1.1618514470890695-0j)
0.5 5j (1.1282341530696185-0j)
0.5 6j (1.101670172921876-0j)
0.5 7j (1.07968319439194-0j)
0.5 8j (1.060882756915604-0j)
0.5 9j (1.0444144598373553-0j)
0.5 10j (1.0297169043542214-0j)
0.75 (-0+0j) (2.016870144382574-0j)
0.75 0.11549619760378992j (1.889378324698205-0j)
0.75 0.24609503336885402j (1.782338002407032-0j)
0.75 0.39634896722901447j (1.690048227686483-0j)
0.75 0.5732404646099698j (1.6083149767721998-0j)
0.75 0.7882938150967403j (1.5337799989444552-0j)
0.75 1.0626359889199166j (1.4633068681246857-0j)
0.75 1.4419435745072013j (1.393022978657784-0j)
0.75 2.060488851853035j (1.3152869353529506-0j)
0.75 4j (1.1867173808403386-0j)
0.75 5j (1.1479369524631808-0j)
0.75 6j (1.117701257248253-0j)
0.75 7j (1.093027480896482-0j)
0.75 8j (1.072209564576019-0j)
0.75 9j (1.0541875297115382-0j)
0.75 10j (1.0382635856077178-0j)
1 (-0+0j) (2.1766781627937255-0j)
1 0.11549619760378992j (2.019796324746499-0j)
1 0.24609503336885402j (1.8901031679825437-0j)
1 0.39634896722901447j (1.7798702893972032-0j)
1 0.5732404646099698j (1.683504963718268-0j)
1 0.7882938150967403j (1.5966251021016638-0j)
1 1.0626359889199166j (1.5152702124910316-0j)
1 1.4419435745072013j (1.434770356175203-0j)
1 2.060488851853035j (1.3463511446680632-0j)
1 4j (1.202226435074913-0j)
1 5j (1.1597254075785202-0j)
1 6j (1.1270197326278528-0j)
1 7j (1.100610733684776-0j)
1 8j (1.0784449491499233-0j)
1 9j (1.0587475535022928-0j)
1 10j (1.039033126165288-0j)
1.25 (-0+0j) (2.310571943737493-0j)
1.25 0.11549619760378992j (2.1281562274182804-0j)
1.25 0.24609503336885402j (1.9790200000215687-0j)
1.25 0.39634896722901447j (1.8534694339918245-0j)
1.25 0.5732404646099698j (1.7445966553858292-0j)
1.25 0.7882938150967403j (1.6470922414179185-0j)
1.25 1.0626359889199166j (1.5562923405030396-0j)
1.25 1.4419435745072013j (1.466912202616366-0j)
1.25 2.060488851853035j (1.3694182384833844-0j)
1.25 4j (1.2133206067469728-0j)
1.25 5j (1.168287307237172-0j)
1.25 6j (1.1339850442567125-0j)
1.25 7j (1.1059611857423473-0j)
1.25 8j (1.0754438150789722-0j)
1.25 9j (0.9534697466795119-0j)
1.25 10j (0.24314206352145817-0j)
1.5 (-0+0j) (2.424831129251398-0j)
1.5 0.11549619760378992j (2.2199794468526592-0j)
1.5 0.24609503336885402j (2.053869659591582-0j)
1.5 0.39634896722901447j (1.9149466965748783-0j)
1.5 0.5732404646099698j (1.7951046132354533-0j)
1.5 0.7882938150967403j (1.6882275420765656-0j)
1.5 1.0626359889199166j (1.5890784002897118-0j)
1.5 1.4419435745072013j (1.491913544676751-0j)
1.5 2.060488851853035j (1.3866695569048224-0j)
1.5 4j (1.220816888821978-0j)
1.5 5j (1.1737416340636977-0j)
1.5 6j (1.1380705354124914-0j)
1.5 7j (1.0858355866594271-0j)
1.5 8j (0.4545076367405072-0j)
1.5 9j (-11.634037430759726-0j)
1.5 10j (-156.7947699228019-0j)
1.75 (-0+0j) (2.5236741318287677-0j)
1.75 0.11549619760378992j (2.2989401072353988-0j)
1.75 0.24609503336885402j (2.1178397607266812-0j)
1.75 0.39634896722901447j (1.9670851622475714-0j)
1.75 0.5732404646099698j (1.837504682103869-0j)
1.75 0.7882938150967403j (1.7223013126132292-0j)
1.75 1.0626359889199166j (1.6157840879247674-0j)
1.75 1.4419435745072013j (1.5118719094162612-0j)
1.75 2.060488851853035j (1.4001264163934746-0j)
1.75 4j (1.226528305681122-0j)
1.75 5j (1.1780325490859704-0j)
1.75 6j (1.1416898652456828-0j)
1.75 7j (0.19789229089871055-0j)
1.75 8j (-39.60468168099871-0j)
1.75 9j (-1400.0749611571848-0j)
1.75 10j (-31037.59840238711-0j)
2.0 (-0+0j) (2.610172213255123-0j)
2.0 0.11549619760378992j (2.367678791042069-0j)
2.0 0.24609503336885402j (2.173202199844841-0j)
2.0 0.39634896722901447j (2.0118587629130396-0j)
2.0 0.5732404646099698j (1.873542006286154-0j)
2.0 0.7882938150967403j (1.7508850666635987-0j)
2.0 1.0626359889199166j (1.6378330488774056-0j)
2.0 1.4419435745072013j (1.5280389536392653-0j)
2.0 2.060488851853035j (1.4107585249066994-0j)
2.0 4j (1.2307416729429386-0j)
2.0 5j (1.182398828445498-0j)
2.0 6j (1.130757876492569-0j)
2.0 7j (-42.674798974477-0j)
2.0 8j (-2693.745970866763-0j)
2.0 9j (-156106.49458678713-0j)
2.0 10j (-5079630.785632959-0j)
2.25 (-0+0j) (2.686606388120583-0j)
2.25 0.11549619760378992j (2.428159285238066-0j)
2.25 0.24609503336885402j (2.221667295048251-0j)
2.25 0.39634896722901447j (2.0507899272512353-0j)
2.25 0.5732404646099698j (1.9046092558368135-0j)
2.25 0.7882938150967403j (1.775283435611127-0j)
2.25 1.0626359889199166j (1.6564601170174782-0j)
2.25 1.4419435745072013j (1.5415711914954777-0j)
2.25 2.060488851853035j (1.4196119284206896-0j)
2.25 4j (1.2339912424043784-0j)
2.25 5j (1.1844077715987986-0j)
2.25 6j (0.09170496512550937-0j)
2.25 7j (-1692.2786667551125-0j)
2.25 8j (-152883.31361596618-0j)
2.25 9j (-16164380.397114296-0j)
2.25 10j (-966295002.5623425-0j)
2.5 (-0+0j) (2.7546634841676916-0j)
2.5 0.11549619760378992j (2.481798555506556-0j)
2.5 0.24609503336885402j (2.2644354938860354-0j)
2.5 0.39634896722901447j (2.0849117012754617-0j)
2.5 0.5732404646099698j (1.9316083892250235-0j)
2.5 0.7882938150967403j (1.7962842413171993-0j)
2.5 1.0626359889199166j (1.6723333279297004-0j)
2.5 1.4419435745072013j (1.5529885536143937-0j)
2.5 2.060488851853035j (1.4270060359775203-0j)
2.5 4j (1.2353809060782537-0j)
2.5 5j (1.2790639540829232-0j)
2.5 6j (35.68802535850027-0j)
2.5 7j (-64108.24956651612-0j)
2.5 8j (-10580734.82573503-0j)
2.5 9j (-1911316524.7307162-0j)
2.5 10j (-151594742093.66263-0j)
2.75 (-0+0j) (2.8157465292819515-0j)
2.75 0.11549619760378992j (2.5297727036934323-0j)
2.75 0.24609503336885402j (2.3025065832061635-0j)
2.75 0.39634896722901447j (2.1150906454059593-0j)
2.75 0.5732404646099698j (1.955301869714471-0j)
2.75 0.7882938150967403j (1.8145581408527165-0j)
2.75 1.0626359889199166j (1.6860284267681984-0j)
2.75 1.4419435745072013j (1.5627581204906005-0j)
2.75 2.060488851853035j (1.433278586135805-0j)
2.75 4j (1.212899394346786-0j)
2.75 5j (1.5297813948617922-0j)
2.75 6j (-514.6971024945163-0j)
2.75 7j (-2496491.188248904-0j)
2.75 8j (-716929921.8581233-0j)
2.75 9j (-189006725718.40887-0j)
2.75 10j (-30584663856540.914-0j)
3.0 (-0+0j) (2.8709247190516445-0j)
3.0 0.11549619760378992j (2.57297475541565-0j)
3.0 0.24609503336885402j (2.3366399289058832-0j)
3.0 0.39634896722901447j (2.141986300164573-0j)
3.0 0.5732404646099698j (1.9762696052536521-0j)
3.0 0.7882938150967403j (1.8306115376036183-0j)
3.0 1.0626359889199166j (1.6979739925442108-0j)
3.0 1.4419435745072013j (1.5712211462926957-0j)
3.0 2.060488851853035j (1.4386729631677917-0j)
3.0 4j (1.0176397401233703-0j)
3.0 5j (-8.969463820153965-0j)
3.0 6j (9976.831623967033-0j)
3.0 7j (-88861524.44858418-0j)
3.0 8j (-42209792546.403015-0j)
3.0 9j (-19367241828454.32-0j)
3.0 10j (-4632387673149303-0j)

In [ ]:
err_01_30_unequal = [1.1311751600096585, 0.8577529474889062, 0.7108553850094026, 0.5421325375843897, 0.3486225019596564, 0.12529382174786363, 0.1353712131956525, 0.44456862811469805, 0.8195430473330383, 1.288422824713532]
err_005_30_unequal = [0.8440508388262805, 0.8556747330993618, 0.7075284773107623, 0.5377527440585371, 0.34330695510714043, 0.11911077288068643, 0.1423990297010631, 0.45247442557929257, 0.8284437869366876, 1.2985794627271225]

err_01_15_unequal = [0.9855090765088775, 0.8844668469373481, 0.767716826687409, 0.6361831628487614, 0.4883006506016789, 0.32108396010897594, 0.12989387324617943, 0.09226652037648772, 0.3560294413935278, 0.6779327752319176]
err_005_15_unequal = [0.9851537142074075, 0.8827277029797143, 0.7649494590017487, 0.6325576942045754, 0.4839259520351142, 0.316030484364724, 0.12419795015378604, 0.09860770421981839, 0.36307235013011085, 0.6858169682072057]

err_01_05_unequal =  [0.9911894555960662, 0.9313822080896308, 0.8642713503253541, 0.7905415188386972, 0.709633056460113, 0.6203807406508636, 0.5209953872500139, 0.4088648395197435, 0.2801807814770843, 0.1293046062362685]
err_005_05_unequal = [0.990964464579642, 0.9302801014721724, 0.8625147108898402, 0.7882379655048518, 0.7068527261954584, 0.6171721132526407, 0.5173902790254119, 0.40487881704351897, 0.27580996399976726, 0.12451783616077883]


In [ ]:
x

In [ ]:
x = -np.log(quad_points)

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(21, 6))
ax1.semilogy(x, err_005_05_unequal, '^-')
ax1.semilogy(x, err_01_05_unequal, 'o-')
ax1.legend([ 'Error ($h = 0.05$)', 'Error ($h = 0.1$)'], fontsize = 15)

ax1.set_xlabel('$1\mathrm{i}*k$', fontsize = 15)
ax1.set_ylabel('absolute error', fontsize = 15)
ax1.set_title('$Z = 0.5$', fontsize = 15)
    
ax2.semilogy(x, err_005_15_unequal, '^-')
ax2.semilogy(x, err_01_15_unequal, 'o-')
ax2.legend([ 'Error ($h = 0.05$)', 'Error ($h = 0.1$)'], fontsize = 15)

ax2.set_xlabel('$1\mathrm{i}*k$', fontsize = 15)
ax2.set_ylabel('absolute error', fontsize = 15)
ax2.set_title('$Z = 1.5$', fontsize = 15)

ax3.semilogy(x, err_005_30_unequal, '^-')
ax3.semilogy(x, err_01_30_unequal, 'o-')
ax3.legend([ 'Error ($h = 0.05$)', 'Error ($h = 0.1$)'], fontsize = 15)

ax3.set_xlabel('$1\mathrm{i}*k$', fontsize = 15)
ax3.set_ylabel('absolute error', fontsize = 15)
ax3.set_title('$Z = 3.0$', fontsize = 15)

fig.tight_layout()

fig.suptitle('Relative error between the estimated integrand value and asymptotic series in two spheres with unequal radii case', fontsize = 20)
fig.subplots_adjust(top=0.85)
fig.savefig('rel_err_unequal.pdf')